Clean Landsat composites created with the file "download_landsat_composite.ipynb"

1. remove high values (1.000000e+20) and replace with NAN
2. add spectral indices

In [3]:
# Import general libraries
import rasterio
import numpy as np
import rasterio

from pathlib import Path

# Import EOdal libraries
from eodal.core.raster import RasterCollection

# define the year/years for the download
years = list(range(2017, 2024)) # from year to year, 1984 is the first available
excluded_years = [1988, 1992, 1993, 1996, 1997, 2022] #no scenes available for this years in the USGS archive.
filtered_years = [year for year in years if year not in excluded_years]

for year in filtered_years: 

     # Path to the raw input TIFF file
    input_tif_path = f'S:\MSc_23_TimckeFinn\data\EOdal\landsat_median_composite_{year}_noetm.tif'

    # Path to the clean output TIFF file
    output_tif_path = f'S:\MSc_23_TimckeFinn\data\EOdal\landsat_median_composite_{year}_noetm_clean.tif'

    # Open the TIFF file using rasterio
    with rasterio.open(input_tif_path) as src:

        # Read all bands of the raster data
        raster_data = src.read()

        # Replace specific values (e.g., 1e+20) with NaN for each band
        replaced_data = np.where(raster_data == 1e+20, np.nan, raster_data)

        # Create a new raster file with NaN values
        with rasterio.open(
            output_tif_path,
            'w',
            driver='GTiff',
            height=src.height,
            width=src.width,
            count=src.count,  # Number of bands
            dtype=replaced_data.dtype,
            crs=src.crs,
            transform=src.transform,
        ) as dst:
            dst.write(replaced_data)

    # read in clean tif again and add some SI

    #read tif from disk:
    fpath_raster = Path(f'S:\MSc_23_TimckeFinn\data\EOdal\landsat_median_composite_{year}_noetm_clean.tif')

    # initialize a RasterCollection from the .tif file
    composite = RasterCollection.from_multi_band_raster(fpath_raster=fpath_raster,
                                                        band_names_dst= ["blue_median", "green_median", "red_median", "nir08_median", "swir16_median","swir22_median"],
                                                        band_aliases = ["blue", "green", "red", "nir_1", "swir_1","swir_2"])

    # calculate some SI
    composite.calc_si('NDVI', inplace = True)
    composite.calc_si('NDWI', inplace = True)
    composite.calc_si('EVI', inplace = True)
    composite.calc_si('CI_GREEN', inplace = True)
    composite.calc_si('MSAVI', inplace = True)
    composite.calc_si('BSI', inplace = True)

    # save as GeoTiff
    composite.to_rasterio(f'S:\MSc_23_TimckeFinn\data\EOdal\landsat_median_composite_{year}_noetm_clean_si.tif')
